In [1]:
import pandas as pd
import numpy as np

Датасет объявлений на Авито

In [61]:
train = pd.read_csv('data/avito_train.csv')
test = pd.read_csv('data/avito_test.csv')

In [62]:
train.head()

,title,description,Category_name,Category
0,"Эбу Renault / Nissan 1,5 DCI Delphi 2001-2008 год","Комплект ЭБУ (мозги, компьютер мотора, двигате...",Запчасти и аксессуары,10
1,Утюг утп 1000 ватт СССР 1987 год,Продам/\n Фото № 1-2 /\n /\nУтюг УТП 1000 ватт...,Бытовая техника,21
2,Возвму машину с выкупом,Возьму машину в аренду с последующим выкупом н...,Предложение услуг,114
3,Полусапожки,"полусапожки в отличном состоянии, один раз оде...","Одежда, обувь, аксессуары",27
4,Босоножки кожаные,"Кожаные(натур) босоножки Karlo Pasolini, 40 рр...","Одежда, обувь, аксессуары",27


In [63]:
train.shape

(4234042, 4)

Проверяем пропуски

In [64]:
train.isnull().sum()

title            0
description      2
Category_name    0
Category         0
dtype: int64

Заполняем их пустой строкой

In [67]:
train = train.fillna("")
train['title&description'] = train['title']+' '+train.description
train.Category.nunique() #количество объектов для предсказания

50

Чтобы извлечь признаки из текстовых даннных воспользуемся векторизатором CountVectorizer

In [291]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize

try_train = train.head(10000) #возьмем первые 10к строк

try_vec = CountVectorizer()
try_bow = try_vec.fit_transform(np.array(try_train['title&description'])) #ранее объединенный двумя признаками столбец

Будем удалять из текста русские "стопслова", которые не несут смысла и замедляют пересорт

In [204]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize

print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

Применим стемминг - отбросим аффиксы (окончания и суффиксы) у слов и приведем их всех к норме

In [237]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('russian')

Кастомный токенайзер - он перебирает слова, удаляет стопслова и начинющиеся на цифры и символы и проводит стемминг

In [293]:
superiority = [' '.join([stemmer.stem(w) for w in wordpunct_tokenize(t.lower()) if (w.isalpha() and w not in stopwords.words('russian'))]) for t in try_train['title&description']]

In [ ]:
from collections import OrderedDict

Попытки ускорить процесс

In [318]:
# superiority = [' '.join([stemmer.stem(w) for w in list(set(wordpunct_tokenize(t.lower()))) if (w.isalpha() and w not in stopwords.words('russian'))]) for t in try_train['title&description']]
# superiority = [' '.join([stemmer.stem(w) for w in list(OrderedDict.fromkeys(wordpunct_tokenize(t.lower()))) if (w.isalpha() and w not in stopwords.words('russian'))]) for t in try_train['title&description']]
# superiority = [' '.join([stemmer.stem(w) for w in wordpunct_tokenize(t.lower()) if (w.isalpha() and stopwords.words('russian').count(w) < 1)]) for t in try_train['title&description']]
superiority = [' '.join([stemmer.stem(w) for w in wordpunct_tokenize(t.lower()) if (w.isalpha() and w not in np.array(stopwords.words('russian')))]) for t in try_train['title&description']]

In [303]:
from sklearn.model_selection import train_test_split

In [298]:
super_vec = CountVectorizer()
super_bow = super_vec.fit_transform(superiority)
x_train, x_test, y_train, y_test = train_test_split(super_bow, try_train.Category, test_size=0.2, random_state=1)

In [299]:
super_bow

<10000x27399 sparse matrix of type '<class 'numpy.int64'>'
	with 263627 stored elements in Compressed Sparse Row format>

Обучим данные на векторизаторах CountVectorizer и Tfidf с использованием SGDClassifier и посчитаем точность

In [305]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

sgd = SGDClassifier(loss='modified_huber', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)

sgd.fit(x_train, y_train)

sgd_y_pred = sgd.predict(x_test)

accuracy_score(y_test, sgd_y_pred)

0.7148

Слова, встречающиеся в одном тексте и сильно реже в других, обладают наибольшей ценностью для данного текста, ровно как и слова, которые часто встречаются в каждом тексте. Для разделения таких слов можно использовать статистическую меру TF-IDF (term frequency–inverse document frequency), характеризующую важность слова для конкретного текста. Для каждого слова из текста $d$ рассчитывается относительная частота встречаемости в нем (Term Frequency):

$$
\text{TF}(t, d) = \frac{C(t | d)}{\sum\limits_{k \in d}C(k | d)},
$$
где $C(t | d)$ - число вхождений слова $t$ в текст $d$.

In [313]:
from sklearn.feature_extraction.text import TfidfVectorizer

super_vec = TfidfVectorizer(max_df=0.9)
super_bow = super_vec.fit_transform(superiority)
x_train, x_test, y_train, y_test = train_test_split(super_bow, try_train.Category, test_size=0.2, random_state=1)

sgd = SGDClassifier(loss='modified_huber', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)

sgd.fit(x_train, y_train)

sgd_y_pred = sgd.predict(x_test)

accuracy_score(y_test, sgd_y_pred)

0.7655

Применение метода hashing trick, который случайно группирует признаки и складывает их

In [316]:
from sklearn.feature_extraction.text import HashingVectorizer

super_vec = HashingVectorizer(n_features=30000)
super_bow = super_vec.fit_transform(superiority)
x_train, x_test, y_train, y_test = train_test_split(super_bow, try_train.Category, test_size=0.2, random_state=1)

sgd = SGDClassifier(loss='modified_huber', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)

sgd.fit(x_train, y_train)

sgd_y_pred = sgd.predict(x_test)

accuracy_score(y_test, sgd_y_pred)

0.748